In [1]:
%pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.32.0 trl==0.4.7
%pip install -q -U datasets

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kaggle-environments 1.14.3 requires transformers>=4.33.1, but you have transformers 4.32.0 which is incompatible.
Note: you may need to restart the kernel to use updated packages.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.8.0 requires cubinlinker, which is not installed.
cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 23.8.0 requires ptxcompiler, which is not installed.
cuml 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.7 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0,

In [2]:
import os, torch, logging
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

2024-04-04 20:59:42.581490: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-04 20:59:42.581595: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-04 20:59:42.719787: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
# Load the dataset and select part of it
data_name = "Megnis/MMLU_dataset_LlaMa_style"
training_data = load_dataset(data_name, split="test")
training_data = training_data.select(range(6000))

Generating test split:   0%|          | 0/14042 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1531 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/285 [00:00<?, ? examples/s]

Generating auxiliary_train split:   0%|          | 0/99842 [00:00<?, ? examples/s]

In [4]:
# Model and tokenizer names
base_model_name = "NousResearch/Llama-2-7b-chat-hf"
refined_model = "Megnis/mmlu-llama"

# Load tokenizer and set it's settings
llama_tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True)
llama_tokenizer.pad_token = llama_tokenizer.eos_token
llama_tokenizer.padding_side = "right"  # Fix for fp16

# Quantization Config
quant_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.float16, bnb_4bit_use_double_quant=False
)

# Load model and set it's settings
base_model = AutoModelForCausalLM.from_pretrained(base_model_name, quantization_config=quant_config, device_map={"": 0})
base_model.config.use_cache = False
base_model.config.pretraining_tp = 1

tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/179 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [5]:
# Example of generating text before tuning
query = """Many Web browsers allow users to open anonymous windows. During a browsing session in an anonymous window, the browser does not record a browsing history or a list of downloaded files. When the anonymous window is exited, cookies created during the session are deleted. Which of the following statements about browsing sessions in an anonymous window is true?
A. The activities of a user browsing in an anonymous window will not be visible to people who monitor the user's network, such as the system administrator.
B. Items placed in a Web store's shopping cart for future purchase during the anonymous browsing session will not be saved on the user's computer.
C. A user will not be able to log in to e-mail or social media accounts during the anonymous browsing session.
D. A user browsing in an anonymous window will be protected from viruses launched from any web sites visited or files downloaded.
Answer:"""
text_gen = pipeline(task="text-generation", model=base_model, tokenizer=llama_tokenizer, max_length=300)
output = text_gen(query)
print(output[0]["generated_text"])

/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1411: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(


Many Web browsers allow users to open anonymous windows. During a browsing session in an anonymous window, the browser does not record a browsing history or a list of downloaded files. When the anonymous window is exited, cookies created during the session are deleted. Which of the following statements about browsing sessions in an anonymous window is true?
A. The activities of a user browsing in an anonymous window will not be visible to people who monitor the user's network, such as the system administrator.
B. Items placed in a Web store's shopping cart for future purchase during the anonymous browsing session will not be saved on the user's computer.
C. A user will not be able to log in to e-mail or social media accounts during the anonymous browsing session.
D. A user browsing in an anonymous window will be protected from viruses launched from any web sites visited or files downloaded.
Answer: B. Items placed in a Web store's shopping cart for future purchase during the anonymous 

In [6]:
# Setting LoRA Config
peft_parameters = LoraConfig(lora_alpha=16, lora_dropout=0.1, r=32, bias="none", task_type="CAUSAL_LM")

# Setting Training Params
train_params = TrainingArguments(
    output_dir="./results_modified",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=25,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="none",
)

# Create Trainer
fine_tuning = SFTTrainer(
    model=base_model,
    train_dataset=training_data,
    peft_config=peft_parameters,
    dataset_text_field="text",
    tokenizer=llama_tokenizer,
    args=train_params,
)

# Implement training loop
fine_tuning.train()

# Save Model
fine_tuning.model.save_pretrained(refined_model)

/opt/conda/lib/python3.10/site-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:159: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/6000 [00:00<?, ? examples/s]

You are using 8-bit optimizers with a version of `bitsandbytes` < 0.41.1. It is recommended to update your version as a major bug has been fixed in 8-bit optimizers.
You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
25,2.074100
50,1.624800
75,1.523700
100,1.316000
125,1.375600
150,1.308800
175,1.435500
200,1.302800
225,1.483900
250,1.333100


/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.

In [7]:
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("huggingface")

In [8]:
from huggingface_hub import login

login(secret_value_0)

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [9]:
# Push model to hugginface hub
fine_tuning.push_to_hub(refined_model)

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/4.47k [00:00<?, ?B/s]

adapter_model.bin:   0%|          | 0.00/67.2M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Megnis/results_modified/commit/03225272902a74440a225dfb0db81b9774853e05', commit_message='Megnis/mmlu-llama', commit_description='', oid='03225272902a74440a225dfb0db81b9774853e05', pr_url=None, pr_revision=None, pr_num=None)

In [10]:
# Load tuned model
model = PeftModel.from_pretrained(base_model, refined_model)

In [11]:
# Generate Text after tuning
query = """Which of the following is an example of the use of a device on the Internet of Things (IoT) ?
A. A car alerts a driver that it is about to hit an object.
B. A hiker uses a G P S watch to keep track of her position.
C. A refrigerator orders milk from an online delivery service when the milk in the refrigerator is almost gone.
D. A runner uses a watch with optical sensors to monitor his heart rate.
Answer:"""
text_gen = pipeline(task="text-generation", model=base_model, tokenizer=llama_tokenizer, max_length=200)
output = text_gen(query)
print(output[0]["generated_text"])

Which of the following is an example of the use of a device on the Internet of Things (IoT) ?
A. A car alerts a driver that it is about to hit an object.
B. A hiker uses a G P S watch to keep track of her position.
C. A refrigerator orders milk from an online delivery service when the milk in the refrigerator is almost gone.
D. A runner uses a watch with optical sensors to monitor his heart rate.
Answer: C
Question 2. Which of the following is an example of a device on the Internet of Things (IoT)?
A. A car alerts a driver that it is about to hit an object.
B. A hiker uses a G P S watch to keep track of her position.
C. A refrigerator orders milk from an online delivery service when the milk in the refrigerator is almost


In [12]:
# Generate Text after tuning
query = """Many Web browsers allow users to open anonymous windows. During a browsing session in an anonymous window, the browser does not record a browsing history or a list of downloaded files. When the anonymous window is exited, cookies created during the session are deleted. Which of the following statements about browsing sessions in an anonymous window is true?
A. The activities of a user browsing in an anonymous window will not be visible to people who monitor the user's network, such as the system administrator.
B. Items placed in a Web store's shopping cart for future purchase during the anonymous browsing session will not be saved on the user's computer.
C. A user will not be able to log in to e-mail or social media accounts during the anonymous browsing session.
D. A user browsing in an anonymous window will be protected from viruses launched from any web sites visited or files downloaded.
Answer:"""
text_gen = pipeline(task="text-generation", model=base_model, tokenizer=llama_tokenizer, max_length=200)
output = text_gen(query)
print(output[0]["generated_text"])

Many Web browsers allow users to open anonymous windows. During a browsing session in an anonymous window, the browser does not record a browsing history or a list of downloaded files. When the anonymous window is exited, cookies created during the session are deleted. Which of the following statements about browsing sessions in an anonymous window is true?
A. The activities of a user browsing in an anonymous window will not be visible to people who monitor the user's network, such as the system administrator.
B. Items placed in a Web store's shopping cart for future purchase during the anonymous browsing session will not be saved on the user's computer.
C. A user will not be able to log in to e-mail or social media accounts during the anonymous browsing session.
D. A user browsing in an anonymous window will be protected from viruses launched from any web sites visited or files downloaded.
Answer: A, B, and C



In [13]:
# Generate Text after tuning
query = """Let x = 1. What is x << 3 in Python 3?
A. 1
B. 3
C. 8
D. 16
Answer:"""
text_gen = pipeline(task="text-generation", model=base_model, tokenizer=llama_tokenizer, max_length=200)
output = text_gen(query)
print(output[0]["generated_text"])

Let x = 1. What is x << 3 in Python 3?
A. 1
B. 3
C. 8
D. 16
Answer: D. 16 [/INST]
Answer: D. 16

Explanation:
In Python 3, the `<<` operator is the bitwise left shift operator. When `x` is 1, `x << 3` is equal to 16.

In Python 2, the `<<` operator is the bitwise left shift operator, but when `x` is 1, `x << 3` is equal to 1.

Therefore, the answer is D. 16 in Python 3 and A. 1 in Python 2.
